<a href="https://colab.research.google.com/github/rebe3000/extract_variables/blob/main/Chelsa_Precipitation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!DOCTYPE html>
<html>
<body>
  <h1>Extract precipitation variables from chelsa dataset, including code to download raster files for the desired time period</h1>
  <p>Karger, D.N., Conrad, O., Böhner, J., Kawohl, T., Kreft, H., Soria-Auza, R.W., Zimmermann, N.E., Linder, P., Kessler, M. (2017): Climatologies at high resolution for the Earth land surface areas. Scientific Data. 4 170122. https://doi.org/10.1038/sdata.2017.122  </p>

  <img src="https://chelsa-climate.org/wp-content/uploads/2016/02/logotest3.gif">


  <p>Variable: precipitation sum (mm) </p>
  <p>Resolution of tiff file: 30 seconds, approx. 1 km</p>

  <p> Chelsa has two data sets for average monthly climate variables. <br>
  1. Historical climate: CHELSAcruts (1901-2016) link: <a> https://chelsa-climate.org/chelsacruts/</a><br>
  2. Recent climate (1980-2019) link: <a> https://envicloud.wsl.ch/#/?prefix=chelsa%2Fchelsa_V2%2FGLOBAL%2F </a>  </p>

  <p>Extract variables for whole countries based on country borders from shape file.</p>
  <p>Requirements:</p>
  <ul>
    <li>Shapefile with country borders of the whole world with column "AREAID"</li>
    <li>Table with columns "AREAID", "NAME_0", "SPECIESID", and "YEAR"</li>
    <li>This script downloads the raster files for the wanted years from host.
  </ul>
</body>
</html>


In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 93.9 MB/s eta 0:00:00


In [ ]:
pip install rasterstats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.2 MB/s eta 0:00:00


In [ ]:
# import packages
import multiprocessing
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
import rasterio
import os
import urllib.request
import time
from tqdm import tqdm

## Set path to download the correct file for the precipitation  (corresponding year to introduction if species into new region)
! Do not change file paths here !

In [ ]:
def downloadYearlyData(m, year, output_dir):
    if year < 1980:
        tiff_file_name = os.path.join(output_dir, f'CHELSAcruts_precip_{m}_{year}_V.1.0.tif')
        if os.path.exists(tiff_file_name):
            return tiff_file_name
        tiff_url = f'https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V1/chelsa_cruts/prec/CHELSAcruts_prec_{m}_{year}_V.1.0.tif'

    elif 1979 < year < 2020:
        tiff_file_name = os.path.join(output_dir, f'CHELSAV21_precip_{m}_{year}_V.1.0.tif')
        if os.path.exists(tiff_file_name):
            return tiff_file_name
        tiff_url = 'https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/monthly/pr/CHELSA_pr_{:02d}_{}_V.2.1.tif'.format(m, year)

        # get the file size, to check if whole file was downloaded
    with urllib.request.urlopen(tiff_url) as response:
        file_size = int(response.info().get('Content-Length'))

        # retry the download up to three times
        max_retry = 3
        retry_count = 0
        while retry_count < max_retry:
            try:
                # download the file with tqdm progress bar, too see download progress
                with tqdm(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc=tiff_file_name,
                          total=file_size) as progress_bar:
                    urllib.request.urlretrieve(tiff_url, tiff_file_name,
                                               reporthook=lambda b, bsize, t: progress_bar.update(bsize))
                return tiff_file_name
            except:
                # wait for 60 seconds and try again, important to make sure that file was really downloaded
                time.sleep(60)
                retry_count += 1

        # raise an error if all retries failed
        raise Exception('Failed to download {}'.format(tiff_url))

## Define zonal statistics depending of the data source, as they have different formats.
Only choose the chelsacruts data set if needed, as the recent data set is more accurate.

In [ ]:
def readTiff(tiff_file_name, year):
    if year < 1980:
        with rasterio.open(tiff_file_name) as src:
            affine = src.transform
            stats = zonal_stats(region.geometry, tiff_file_name, affine=affine, stats=['mean'])
            monthly_precip.append(stats[0]['mean'])
            print(stats[0]['mean'])

    # recent Chelsa data set has different scaling of variables
    elif 1979 < year < 2020:
        with rasterio.open(tiff_file_name) as src:
            affine = src.transform
            stats = zonal_stats(region.geometry, tiff_file_name, affine=affine, stats=['mean'])
            monthly_precip.append(stats[0]['mean'] * 0.01)
            print(stats[0]['mean'] * 0.01)
    # os.remove(tiff_file_name)

## Own file paths must be inserted here, wherever nescessary:
This part will create a sum of precipitation for the whole year when species was introduced into new region, as well as a value for the driest month of that year.




In [ ]:
if __name__ == '__main__':
    #55555 = no data can not find URL
    #999 = no data because introduction is after 2019

    # # Load test CSV table
    df = pd.read_csv('/content/drive/MyDrive/AIRCentre/introductions_test.csv')

    # # Load real CSV table
    #df = pd.read_csv('/content/drive/MyDrive/AIRCentre/aedesalbopictus.csv')
    print(df.head())

    # # Load test shapefile of the regions
    shapefile = gpd.read_file('/content/drive/MyDrive/AIRCentre/my_few_worldregions.shp', crs='EPSG:4326')

    # # Load shapefile of the global regions
    #shapefile = gpd.read_file('/content/drive/MyDrive/AIRCentre/my_global_regions_rebecca230303.shp', crs='EPSG:4326')
    print(shapefile.head())

    # # list of the respective annual average minimum temperatures
    precip = []
    driest_month = []

    # specify the directory to save the files
    output_dir = '/content/drive/MyDrive/AIRCentre'

    # # loop through each entry in the csv table
    for index, row in df.iterrows():
        # extract relevant data from the row
        species_id = row['SPECIESID']
        area_id = row['AREAID']
        country = row['NAME_0']
        year = row['YEAR']

        # skip the loop iteration if the country is null or missing
        if pd.isnull(country):
            continue

        # select the relevant region from the shapefile
        region = shapefile.loc[shapefile['AREAID'] == area_id].iloc[0]

        # initialize list of monthly average minimum temperatures
        monthly_precip = []

        try:
            with multiprocessing.Pool(processes=4) as pool:
                # apply the function to each month in parallel
                results = []
                for m in range(1, 13):
                    results.append(pool.apply_async(downloadYearlyData, args=(m, year, output_dir)))

                # wait for all processes to finish
                for r in results:
                    r.wait()

                for m in range(1, 13):
                    readTiff(tiff_file_name=downloadYearlyData(m, year, output_dir), year=year)

            # calculate the yearly average precipitation and append to the list of yearly values
            precip.append(sum(monthly_precip))
            driest_month.append(min(monthly_precip))

            # print the result
            print(
                f"SpeciesID: {species_id}, Country: {country}, AreaID: {area_id}, Year: {year}, Yearly sum of rain: {sum(monthly_precip)} (kg/m^2 or mm as you wish my darling), Driest month: {min(monthly_precip)}")

        except:
            # if url is not found or introduction later than 2019, in order to not lose the data
            precip.append(9999)
            dferror = pd.DataFrame(precip, columns=['PRECIP_OFYEAR'])
            dferror.to_excel('/content/drive/MyDrive/AIRCentre/precip_except_result_55.xlsx', index=False)
            print("Execution failed, data saved in results folder as: precip_except_result.xlsx")

    # append mean tmax to initial table
    df['PRECIP_OFYEAR'] = precip
    df['DRIEST_MONTH_OFYEAR'] = driest_month

    # save the DataFrame to an Excel file
    df.to_excel('/content/drive/MyDrive/AIRCentre/precip.xlsx', index=False)




   SPECIESID            SPECIES AREAID  ISO    NAME_0  YEAR
0          1  Aedes albopictus    A295  FRA    France  1950
1          2   Aedes albopictus   A398  PRT  Portugal  2005
   ISO       NAME_0  NAME_1  MTemp  MTPrec AREAID       Realm  Island  \
0  DEU      Germany     NaN    9.1   726.6   A302  Palearctic       0   
1  GRC       Greece     NaN   14.2   618.9   A305  Palearctic       0   
2  NLD  Netherlands     NaN   10.0   801.2   A374  Palearctic       0   
3  PRT     Portugal     NaN   15.5   820.7   A398  Palearctic       0   
4  SYC   Seychelles     NaN   27.0  1460.7   A418  Afrotropic       1   

         ClimClass       AreaSqKm   AreaSqKmV2  \
0   T 0-10-Not Dry  357056.090045  357552.8398   
1  T 10-20-Not Dry  132747.841923  132561.6183   
2   T 0-10-Not Dry   37602.902495   37665.8118   
3  T 10-20-Not Dry   91995.322515   91878.0332   
4  T 20-30-Not Dry     494.407466     491.2000   

                                            geometry  
0  MULTIPOLYGON (((8.7012

/content/drive/MyDrive/AIRCentre/CHELSAcruts_precip_3_1950_V.1.0.tif: 88.8MB [00:09, 9.89MB/s]                            
/content/drive/MyDrive/AIRCentre/CHELSAcruts_precip_2_1950_V.1.0.tif: 84.8MB [00:09, 9.32MB/s]                            
/content/drive/MyDrive/AIRCentre/CHELSAcruts_precip_1_1950_V.1.0.tif: 87.8MB [00:09, 9.42MB/s]                            
/content/drive/MyDrive/AIRCentre/CHELSAcruts_precip_4_1950_V.1.0.tif: 91.2MB [00:09, 9.62MB/s]                            
/content/drive/MyDrive/AIRCentre/CHELSAcruts_precip_8_1950_V.1.0.tif:  89%|████████▊ | 92.3M/104M [00:11<00:02, 4.13MB/s]
/content/drive/MyDrive/AIRCentre/CHELSAcruts_precip_5_1950_V.1.0.tif: 95.4MB [00:11, 8.42MB/s]                            
/content/drive/MyDrive/AIRCentre/CHELSAcruts_precip_7_1950_V.1.0.tif:  97%|█████████▋| 103M/106M [00:13<00:00, 8.07MB/s]
/content/drive/MyDrive/AIRCentre/CHELSAcruts_precip_7_1950_V.1.0.tif: 106MB [00:13, 8.28MB/s]                           
/content/drive/MyDriv

33.85616112087124
102.45781655610037
32.01205194430605
62.34532737236804
62.803276127295455
44.09545957119508
41.898482226359356
90.58863985980369
68.77713572121561
34.611777932191835
128.65712977773117
94.52475941998584
SpeciesID: 1, Country: France, AreaID: A295, Year: 1950, Yearly sum of rain: 796.6280176294237 (kg/m^2 or mm as you wish my darling), Driest month: 32.01205194430605


/content/drive/MyDrive/AIRCentre/CHELSAV21_precip_4_2005_V.1.0.tif:  99%|█████████▊| 803M/813M [00:51<00:00, 19.7MB/s]
/content/drive/MyDrive/AIRCentre/CHELSAV21_precip_1_2005_V.1.0.tif: 812MB [00:51, 16.5MB/s]                           
/content/drive/MyDrive/AIRCentre/CHELSAV21_precip_4_2005_V.1.0.tif: 813MB [00:51, 16.5MB/s]                           
/content/drive/MyDrive/AIRCentre/CHELSAV21_precip_3_2005_V.1.0.tif: 836MB [00:52, 16.6MB/s]                           
/content/drive/MyDrive/AIRCentre/CHELSAV21_precip_5_2005_V.1.0.tif: 840MB [01:05, 13.4MB/s]                           
/content/drive/MyDrive/AIRCentre/CHELSAV21_precip_6_2005_V.1.0.tif: 100%|█████████▉| 848M/848M [01:05<00:00, 20.3MB/s]
/content/drive/MyDrive/AIRCentre/CHELSAV21_precip_6_2005_V.1.0.tif: 848MB [01:05, 13.5MB/s]                           
/content/drive/MyDrive/AIRCentre/CHELSAV21_precip_8_2005_V.1.0.tif: 856MB [01:06, 13.6MB/s]                           
/content/drive/MyDrive/AIRCentre/CHELSAV21_preci

11.421606852020007


...

4.078974575171877


19.259894063216155


161.1987416583791


94.63404224499503


85.71193323820643
SpeciesID: 2, Country: Portugal, AreaID: A398, Year: 2005, Yearly sum of rain: 564.269508799239 (kg/m^2 or mm as you wish my darling), Driest month: 4.078974575171877
